In [34]:
import time
import os
import csv
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm


In [26]:

def data_extractor(base_url, retries=3, delay=5):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }

    diagnosis_treatment_link = ""
    doctors_departments_link = ""

    for attempt in range(retries):
        try:
            response = requests.get(base_url, headers=headers, timeout=20)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract Diagnosis & Treatment link
            content1 = soup.find('a', id="et_genericNavigation_diagnosis-treatment")
            if not content1:
                # fallback: search by link text containing both words
                for a in soup.find_all('a'):
                    link_text = a.get_text(separator=' ').strip().lower()
                    if "diagnosis" in link_text and "treatment" in link_text:
                        content1 = a
                        break
            if content1:
                href1 = content1.get('href')
                diagnosis_treatment_link = f"https://www.mayoclinic.org{href1}" if href1 and href1.startswith("/") else href1

            # Extract Doctors & Departments link
            content2 = soup.find('a', id="et_genericNavigation_doctors-departments")
            if not content2:
                # fallback: search by link text containing both words
                for a in soup.find_all('a'):
                    link_text = a.get_text(separator=' ').strip().lower()
                    if "doctors" in link_text and "departments" in link_text:
                        content2 = a
                        break
            if content2:
                href2 = content2.get('href')
                doctors_departments_link = f"https://www.mayoclinic.org{href2}" if href2 and href2.startswith("/") else href2

            break  # success, exit retry loop

        except requests.exceptions.RequestException as e:
            print(f"[Attempt {attempt + 1}] Error fetching {base_url}: {e}")
            if attempt < retries - 1:
                time.sleep(delay)

    return diagnosis_treatment_link, doctors_departments_link

def web_scraping(base_url):
    # Define the expected headers in order
    expected_headers = ["disease", "main_link", "Diagnosis_treatment_link", "Doctors_departments_link"]
    
    # Check if file exists and read existing headers if it does
    file_exists = os.path.isfile("mayo_diseases.csv")
    existing_headers = []
    
    if file_exists:
        with open("mayo_diseases.csv", "r", encoding="utf-8") as file:
            reader = csv.reader(file)
            existing_headers = next(reader, [])
    
    # Determine if we need to write headers
    write_headers = not file_exists or existing_headers != expected_headers
    
    # Get the webpage content
    response = requests.get(base_url)
    if response.status_code != 200:
        print("Failed to retrieve page")
        exit()

    soup = BeautifulSoup(response.text, "html.parser")
    items = soup.select(".cmp-results-with-primary-name__see-link, .cmp-results-with-primary-name a")

    with open("mayo_diseases.csv", "a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        
        # Write headers if needed
        if write_headers:
            writer.writerow(expected_headers)
        
        for item in tqdm(items, desc="Scraping Diseases"):
            disease_name = item.text.strip()
            main_link = f"https://www.mayoclinic.org{item['href']}" if item['href'].startswith("/") else item['href']

            link1, link2 = data_extractor(main_link)
            
            # Create a row with all expected columns
            row_data = {
                "disease": disease_name,
                "main_link": main_link,
                "Diagnosis_treatment_link": link1,
                "Doctors_departments_link": link2
            }
            
            # If appending to existing file with different headers, align data with existing headers
            if file_exists and existing_headers:
                row = [row_data.get(header, "") for header in existing_headers]
            else:
                row = [row_data[header] for header in expected_headers]
            
            writer.writerow(row)

    print("Scraping Completed! Data Saved")

# Example usage:
# web_scraping("https://www.mayoclinic.org/diseases-conditions")

In [27]:
base_url = f"https://www.mayoclinic.org/diseases-conditions/index?letter=A"
web_scraping(base_url) 

Scraping Diseases: 100%|██████████| 132/132 [00:45<00:00,  2.92it/s]

Scraping Completed! Data Saved


In [35]:
from bs4 import BeautifulSoup


base_url="https://www.mayoclinic.org/diseases-conditions/hyperhidrosis/symptoms-causes/syc-20367152"
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
response = requests.get(base_url, headers=headers, timeout=20)
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')
extraction_list = ["overview","symptoms","when-to-see-a-doctor","causes","complications","prevention","risk-factors"]

for i in extraction_list:
# Find the overview section by its aria-labelledby attribute
   overview_section = soup.find('section', {'aria-labelledby': i})

# Extract all paragraph text from the overview content
   overview_content = overview_section.find('div', class_='cmp-text__rich-content')
   overview_paragraphs = [p.get_text() for p in overview_content.find_all('p')]

# Join paragraphs into a single string
   overview_text = '\n\n'.join(overview_paragraphs)

   print(overview_text)

AttributeError: 'NoneType' object has no attribute 'find'

In [37]:
from bs4 import BeautifulSoup

def extract_overview(html_content):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    response = requests.get(base_url, headers=headers, timeout=20)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the Overview section - looking for h2 with text "Overview"
    overview_header = soup.find(lambda tag: tag.name == 'h2' and 'when-to-see-a-doctor"' in tag.text)
    
    if not overview_header:
        return "Overview section not found"
    
    # Initialize overview paragraphs
    overview_paragraphs = []
    
    # Get all siblings after the h2 until the next h2
    for sibling in overview_header.find_next_siblings():
        if sibling.name == 'h2':
            break  # Stop when we reach the next section
        if sibling.name == 'p':
            overview_paragraphs.append(sibling.get_text(strip=True))
    
    if not overview_paragraphs:
        return "No overview content found"
    
    return '\n\n'.join(overview_paragraphs)

# Example usage:
base_url="https://www.mayoclinic.org/diseases-conditions/hyperhidrosis/symptoms-causes/syc-20367152"

html_content = """ (your HTML content here) """
overview_text = extract_overview(base_url)
print(overview_text)

Overview section not found


In [43]:
from bs4 import BeautifulSoup
import requests

def extract_mayo_clinic_sections(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        main_content = soup.find('div', class_='content')
        if not main_content:
            return {"error": "Main content not found"}
        
        sections = {
            "Overview": None,
            "Symptoms": None,
            "When to see a doctor": None,
            "Causes": None,
            "Risk factors": None,
            "Complications": None,
            "Prevention": None
        }
        
        # Find all headings (h2 and h3)
        headings = main_content.find_all(['h2', 'h3'])
        
        for heading in headings:
            heading_text = heading.get_text(strip=True)
            
            # Handle the typo "dotor" in the HTML
            if "When to see a dotor" in heading_text:
                heading_text = "When to see a doctor"
                
            if heading_text in sections:
                content = []
                next_node = heading.next_sibling
                
                # Collect content until next heading
                while next_node and next_node.name not in ['h2', 'h3']:
                    if next_node.name == 'p':
                        content.append(next_node.get_text(strip=True))
                    elif next_node.name in ['ul', 'ol']:
                        items = [li.get_text(strip=True) for li in next_node.find_all('li')]
                        content.extend(items)
                    next_node = next_node.next_sibling
                
                sections[heading_text] = '\n'.join(content) if content else "No content found"
        
        return {k: v for k, v in sections.items() if v is not None}
    
    except requests.exceptions.RequestException as e:
        return {"error": f"Failed to fetch page: {str(e)}"}

# Example usage:
url = "https://www.mayoclinic.org/diseases-conditions/hyperhidrosis/symptoms-causes/syc-20367152"
sections = extract_mayo_clinic_sections(url)

for section_name, content in sections.items():
    print(f"=== {section_name.upper()} ===")
    print(content)
    print("\n" + "="*50 + "\n")

=== OVERVIEW ===
Hyperhidrosis (hi-pur-hi-DROE-sis) is excessive sweating that's not always related to heat or exercise. You may sweat so much that it soaks through your clothes or drips off your hands. Heavy sweating can disrupt your day and cause social anxiety and embarrassment.
Hyperhidrosis treatment usually helps. It often begins with antiperspirants. If these don't help, you may need to try different medications and therapies. In severe cases, your health care provider may suggest surgery to remove the sweat glands or to disconnect the nerves related to producing too much sweat.
Sometimes an underlying condition may be found and treated.


=== SYMPTOMS ===
The main symptom of hyperhidrosis is heavy sweating. This goes beyond the sweating from being in a hot environment, exercising, or feeling anxious or stressed. The type of hyperhidrosis that usually affects the hands, feet, underarms or face causes at least one episode a week when you're awake. And the sweating usually happens

In [44]:
import csv
from bs4 import BeautifulSoup
import requests
import os

def extract_sections(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        main_content = soup.find('div', class_='content') or soup.find('article', id='main-content')
        if not main_content:
            return {"error": "Main content not found"}
        
        sections = {
            "Overview": None,
            "Symptoms": None,
            "When to see a doctor": None,
            "Causes": None,
            "Risk factors": None,
            "Complications": None,
            "Prevention": None
        }
        
        headings = main_content.find_all(['h2', 'h3'])
        
        for heading in headings:
            heading_text = heading.get_text(strip=True)
            if "When to see a dotor" in heading_text:
                heading_text = "When to see a doctor"
                
            if heading_text in sections:
                content = []
                next_node = heading.next_sibling
                while next_node and next_node.name not in ['h2', 'h3']:
                    if next_node.name == 'p':
                        content.append(next_node.get_text(strip=True))
                    elif next_node.name in ['ul', 'ol']:
                        items = [li.get_text(strip=True) for li in next_node.find_all('li')]
                        content.extend(items)
                    next_node = next_node.next_sibling
                sections[heading_text] = '\n'.join(content) if content else None
        
        return {k: v for k, v in sections.items() if v is not None}
    
    except Exception as e:
        return {"error": f"Error processing {url}: {str(e)}"}

def update_csv_with_sections(csv_file):
    # Read existing data
    rows = []
    existing_headers = []
    
    if os.path.exists(csv_file):
        with open(csv_file, 'r', newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            existing_headers = reader.fieldnames
            rows = list(reader)
    
    # Define all possible headers
    all_headers = existing_headers.copy() if existing_headers else ['disease', 'main_link']
    section_headers = [
        'Overview', 'Symptoms', 'When to see a doctor', 
        'Causes', 'Risk factors', 'Complications', 'Prevention'
    ]
    
    # Add section headers if not already present
    for header in section_headers:
        if header not in all_headers:
            all_headers.append(header)
    
    # Process each row
    for row in tqdm(rows):
        if 'main_link' in row and row['main_link']:
            print(f"Processing: {row.get('disease', 'Unknown')}")
            sections = extract_sections(row['main_link'])
            
            # Update row with new sections
            for section, content in sections.items():
                if section in all_headers:
                    row[section] = content
                else:
                    print(f"Warning: Unexpected section '{section}' found")
    
    # Write back to CSV
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=all_headers)
        writer.writeheader()
        writer.writerows(rows)
    
    print(f"CSV file updated successfully with {len(rows)} rows")

# Example usage
csv_file = "mayo_diseases.csv"
update_csv_with_sections(csv_file)

  0%|          | 0/132 [00:00<?, ?it/s]

Processing: Atrial fibrillation


  1%|          | 1/132 [00:00<00:49,  2.66it/s]

Processing: Hyperhidrosis


  2%|▏         | 2/132 [00:00<00:48,  2.66it/s]

...

Processing: Avascular necrosis (osteonecrosis)


 99%|█████████▉| 131/132 [00:44<00:00,  2.86it/s]

Processing: Atrioventricular nodal reentry tachycardia (AVNRT)


100%|██████████| 132/132 [00:44<00:00,  2.97it/s]

CSV file updated successfully with 132 rows


In [31]:
extraction_list = ["overview","symptoms","when-to-see-a-doctor","causes","complications","prevention","risk-factors"]